# Supplementary Genome Biology

This is a brief notebook to share examples of applying BASH, R and Python programming to genomic workflows. This is not a step by step command-line procedure.

## 1 Trimmomatic loop

This bash script reduces the manual nature for naming file input and outputs providing better reproducibility for the Trimmomatic software. The thread argument can be used to improve the speed of running with a HPC with multiple cores.

```shell

#!/bin/bash
#SBATCH --job-name=trimmomatic
#SBATCH --output=trimmomatic_out
#SBATCH --error=trimmomatic_error
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --mem=10G
#SBATCH --time=0-01:00:00
#SBATCH --account=bisc029026

#move into your current directory – this will be named in the $SLURM_SUBMIT_DIR environment variable
cd $SLURM_SUBMIT_DIR

# arg1: number of threads
# to run: 
# chmod +x trim.sh
# <path>/trim.sh <number of threads>
# Example: ./trim.sh 40

for f in *_R1.fastq # for each sample

do
    n=${f%%_R1.fastq} # strip part of file name
    java -jar /sw/apps/Trimmomatic-0.39/trimmomatic-0.39.jar PE -threads $1 ${n}_R1.fastq  ${n}_R2.fastq \
    ${n}_R1_trimmed.fastq.gz ${n}_R1_unpaired.fastq.gz ${n}_R2_trimmed.fastq.gz \
    ${n}_R2_unpaired.fastq.gz \
    LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15

done
 ```

Executed as
```shell 
sbatch 01_GBG_trimm_loop.sh 10
```

## 2 Fasta file Biopython Argparser

This python script provides a flexiable way to analyse and filter fasta files by sequence length on the command line without direct filepath calling. Future developments could include gene/protein calling and motif searching.

In [ ]:
# uses the argparse module for better command line
# no direct file paths are requried.

# HPC required modules 
# module add lang/python/3.9.13
# module add lang/python/anaconda/3.7-2019.03.biopython


import argparse
from Bio import SeqIO

# Create  argparse objects and define command line arguments
parser = argparse.ArgumentParser(description='Filter a FASTA file to include only sequences longer than a specified length')
parser.add_argument('input_file', type=str, help='input FASTA file')
parser.add_argument('output_file', type=str, help='output FASTA file')
parser.add_argument('-l', '--length', type=int, default=100, help='minimum length of selected sequences')

# Parse the arguments
args = parser.parse_args()

# Open the input and output files and filter the sequences
count = 0
total = 0
with open(args.input_file, 'r') as input_handle, open(args.output_file, 'w') as output_handle:
    for record in SeqIO.parse(input_handle, 'fasta'):
        total += 1
        if len(record) >= args.length:
            count += 1
            SeqIO.write(record, output_handle, 'fasta')

# Print
print(f'{count} records selected out of {total}')


Which can be executed on the command line as follows.

In [ ]:
python filter_contigs.py -l 1500 final.contigs.fa filtered_contigs_1500.fa

## 3 R - Dplyr to join Prokka and Ghosta Koala data

An example of an R script that joins taxonomic, structural and functional annotations together.
This helps to filter sequences by taxonomy and compare genes between different genera.

```R
library("vroom")
library("tidyverse")
library("ggplot2")

#sets wd to location of rmd file R studio only not console
setwd(dirname(rstudioapi::getActiveDocumentContext()$path))

#load in data from prokka and ghostKoala annotations
prokka_data <- vroom("../GBG_exam/archaea_prokka/PROKKA_04032023.tsv")

ghost_koala <- vroom("../GBG_exam/GhostKoala_archaea/user.out.top")

ghost_annot <- vroom("../GBG_exam/GhostKoala_archaea/user_ko_definition.txt")

#tidy up the names
col_names <- paste0("gtax_Column_", 1:ncol(ghost_koala))
colnames(ghost_koala) <- col_names


col_names <- paste0("gann_Column_", 1:ncol(ghost_annot))
colnames(ghost_annot) <- col_names



#View data
head(prokka_data)
head(ghost_koala)
head(ghost_annot)

#rename useful columns.
ghost_koala <- ghost_koala %>% 
  rename(locus_tag = gtax_Column_1) %>% 
  rename(genus = gtax_Column_5)


ghost_annot <- ghost_annot %>%
  rename(locus_tag = gann_Column_1) %>% 
  rename(KEGG_annotation = gann_Column_3)



# clean up - locus 
ghost_koala <- ghost_koala %>% 
  mutate(locus_tag = str_replace(locus_tag, "user:", ""))


# join the three datasets via locus tag.
joined_df <- inner_join(prokka_data, ghost_koala, by = "locus_tag") %>% 
             inner_join(ghost_annot, by = "locus_tag") %>% 
  select(locus_tag, ftype, genus, gene, EC_number, COG, product , KEGG_annotation, gtax_Column_3, gtax_Column_4)


joined_df

#write output to csv
vroom_write(joined_df, "joined_data.tsv", delim = "\t")

```


## BUSCO 

An extra argument was added from the default to search prokaryotic databases only.

``` shell
busco --in final.contigs.fa --mode genome  --auto-lineage-prok --out busco_results
```

# 4 Quast with read mapping
Quast was run with the reference genome of c.p.syntrophicum, it would be interesting to also explore metaquast for metagenomic analysis. 

``` shell

#### Quast with a reference #####

#download reference genome 
wget --timestamping https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/008/000/775/GCF_008000775.1_ASM800077v1/GCF_008000775.1_ASM800077v1_genomic.fna.gz -P cp_s_genome/

#load quast
module add lang/python/anaconda/3.8-2021-TW

# quast without alignment
quast.py final.contigs.fa -r cp_s_genome/GCF_008000775.1_ASM800077v1_genomic.fna.gz
# run WITH alignment
quast.py final.contigs.fa -r cp_s_genome/GCF_008000775.1_ASM800077v1_genomic.fna.gz --bam sort_cps_align.bam

```

# 5 Read mapping the assembly

To calculate the coverage of reads aligned to the assembly.

In [ ]:
## BWA
module load apps/bwa/0.7.17
module load apps/samtools/1.9

# Index the assembly file
bwa index final.contigs.fa

# Map the reads to the assembly
bwa mem final.contigs.fa coursework2023_R1_trimmed.fastq coursework2023_R2_trimmed.fastq -o cps_align_trimmed.sam

# Convert the SAM file to a BAM file
samtools view -S -b cps_align_trimmed.sam > cps_align_trimmed.bam

# Sort and index the BAM file
samtools sort cps_align_trimmed.bam -o sort_cps_align_trimmed.bam
samtools index sort_cps_align_trimmed.bam

The mean coverage of the BAM file

```shell 
module load apps/samtools/1.9
samtools depth sort_cps_align.bam -a  |  awk '{sum+=$3} END { print "Average = ",sum/NR}'
```

Average =  15.434